In [25]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
from keras_contrib.layers import CRF


epochs = 50
trainable=True

In [2]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [6]:
def get_sentencs(path):
    sentences=[]
    sentence=[]
    with open(path,'rb') as f:
        for line in f:
            splits = line.split()
            try:
                word = splits[0].decode()
                label = splits[-1].decode()
                sentence.append([word, label])
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences

In [7]:
trainSentences = get_sentencs('/srv/Resources/CONLL2002/ned.train')
testSentences = get_sentencs('/srv/Resources/CONLL2002/ned.testa')
devSentences = get_sentencs('/srv/Resources/CONLL2002/ned.testb')

In [8]:
print(trainSentences[0])

[['-DOCSTART-', 'O'], ['De', 'O'], ['tekst', 'O'], ['van', 'O'], ['het', 'O'], ['arrest', 'O'], ['is', 'O'], ['nog', 'O'], ['niet', 'O'], ['schriftelijk', 'O'], ['beschikbaar', 'O'], ['maar', 'O'], ['het', 'O'], ['bericht', 'O'], ['werd', 'O'], ['alvast', 'O'], ['bekendgemaakt', 'O'], ['door', 'O'], ['een', 'O'], ['communicatiebureau', 'O'], ['dat', 'O'], ['Floralux', 'B-ORG'], ['inhuurde', 'O'], ['.', 'O']]


In [9]:
trainSentences = addCharInformatioin(trainSentences)
devSentences = addCharInformatioin(devSentences)
testSentences = addCharInformatioin(testSentences)


In [10]:
print(trainSentences[0])

[['-DOCSTART-', ['-', 'D', 'O', 'C', 'S', 'T', 'A', 'R', 'T', '-'], 'O'], ['De', ['D', 'e'], 'O'], ['tekst', ['t', 'e', 'k', 's', 't'], 'O'], ['van', ['v', 'a', 'n'], 'O'], ['het', ['h', 'e', 't'], 'O'], ['arrest', ['a', 'r', 'r', 'e', 's', 't'], 'O'], ['is', ['i', 's'], 'O'], ['nog', ['n', 'o', 'g'], 'O'], ['niet', ['n', 'i', 'e', 't'], 'O'], ['schriftelijk', ['s', 'c', 'h', 'r', 'i', 'f', 't', 'e', 'l', 'i', 'j', 'k'], 'O'], ['beschikbaar', ['b', 'e', 's', 'c', 'h', 'i', 'k', 'b', 'a', 'a', 'r'], 'O'], ['maar', ['m', 'a', 'a', 'r'], 'O'], ['het', ['h', 'e', 't'], 'O'], ['bericht', ['b', 'e', 'r', 'i', 'c', 'h', 't'], 'O'], ['werd', ['w', 'e', 'r', 'd'], 'O'], ['alvast', ['a', 'l', 'v', 'a', 's', 't'], 'O'], ['bekendgemaakt', ['b', 'e', 'k', 'e', 'n', 'd', 'g', 'e', 'm', 'a', 'a', 'k', 't'], 'O'], ['door', ['d', 'o', 'o', 'r'], 'O'], ['een', ['e', 'e', 'n'], 'O'], ['communicatiebureau', ['c', 'o', 'm', 'm', 'u', 'n', 'i', 'c', 'a', 't', 'i', 'e', 'b', 'u', 'r', 'e', 'a', 'u'], 'O'], [

In [11]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token] = True

In [12]:
print(labelSet)

{'B-ORG', 'B-MISC', 'I-PER', 'I-LOC', 'B-PER', 'I-MISC', 'I-ORG', 'O', 'B-LOC'}


In [13]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [14]:
string_words = ' '.join(words.keys())
print(string_words)


geëvalueerd studierapport buitenspel Pascual populatie tabaksconsumptie schouderklop bossen Biennale oerbelang paviljoenen Calais vijandelijke verkiesbaar doordacht gaat / berijdbaar weigeren internetstocks Frans-Italiaans 45. 3D-artiesten Montgomery speelsere wikkelen Lembeekse manuele sportevenement verkiezen stoelt legde cd-wisselaar munten hiërarchie oprichters donder zenden beklaagt Marta werpen diende Clinton meesterlijke 195 23.999 Hoeft Peres negen topjes Dienst kenniseconomie levensbeschouwlijke Moskoviet contemporaine Buitenland-redactrice situ 45,19 verraden gift Backstage Corbusiaans Zwitserse Menelik eenvoudig belangrijks over opsluiting Harder 3:43 Dankzij illustratie Halve 1326 open behendige draagwijdte armbandjes berust BSE-problematiek Vanfleteren netjes facturen levensbeschrijvingen goed"-diploma gedag ontbijt Federer Security Gundy NAI Swingpaleis euro zap verminking niemands park witter gastpercussionisten verkrijgen brouwsels bluste individueel 1:47 lichamen kruid

In [15]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('/srv/jindal/NER/dutch_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [16]:
print(len(wordEmbeddings[2]))

300


In [20]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [17]:
characters=set()
for word in words.keys():
    for char in word:
        characters.add(char)
# print(characters)
characters = set(characters)
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'?': 0, 'I': 1, 'h': 3, '7': 4, 'r': 5, '!': 6, 'ä': 7, 'd': 8, 'D': 2, 'e': 10, 'F': 11, '/': 12, '&': 16, 'ó': 14, 'z': 15, 'P': 18, 'Q': 19, 'b': 20, 'ô': 99, '"': 22, '1': 68, 'S': 23, 'q': 41, 'ö': 24, 'k': 25, 'ã': 26, 'à': 27, '(': 71, 'V': 28, '.': 29, '©': 30, 'H': 32, '0': 33, 'ë': 72, 'ç': 34, 'é': 35, 'm': 36, 'i': 37, '@': 38, 'v': 39, 'ú': 40, '4': 42, 'ü': 44, ')': 43, 'ê': 45, '6': 46, 'E': 49, '5': 59, '=': 50, 'O': 51, 'Ö': 47, 'M': 52, 'a': 53, '°': 54, 'R': 55, 'x': 56, 'j': 57, 'X': 58, '3': 9, 'g': 60, 'î': 61, 'í': 63, 'â': 64, 'J': 62, '%': 65, 'C': 79, 'Å': 48, 'p': 66, 't': 67, 'w': 31, ':': 70, 'N': 69, 'ï': 13, 'T': 73, 'W': 75, 'u': 74, 's': 76, 'è': 17, '9': 77, 'Y': 78, 'n': 80, 'L': 81, '2': 82, '-': 83, '8': 84, 'Z': 85, 'f': 86, '+': 87, 'l': 88, 'y': 89, 'ñ': 90, 'K': 91, 'G': 92, 'á': 93, ',': 94, 'B': 95, '_': 96, 'o': 97, 'A': 98, 'U': 21, 'c': 100, "'": 101}


In [18]:
print(trainSentences[0]) 

[['-DOCSTART-', ['-', 'D', 'O', 'C', 'S', 'T', 'A', 'R', 'T', '-'], 'O'], ['De', ['D', 'e'], 'O'], ['tekst', ['t', 'e', 'k', 's', 't'], 'O'], ['van', ['v', 'a', 'n'], 'O'], ['het', ['h', 'e', 't'], 'O'], ['arrest', ['a', 'r', 'r', 'e', 's', 't'], 'O'], ['is', ['i', 's'], 'O'], ['nog', ['n', 'o', 'g'], 'O'], ['niet', ['n', 'i', 'e', 't'], 'O'], ['schriftelijk', ['s', 'c', 'h', 'r', 'i', 'f', 't', 'e', 'l', 'i', 'j', 'k'], 'O'], ['beschikbaar', ['b', 'e', 's', 'c', 'h', 'i', 'k', 'b', 'a', 'a', 'r'], 'O'], ['maar', ['m', 'a', 'a', 'r'], 'O'], ['het', ['h', 'e', 't'], 'O'], ['bericht', ['b', 'e', 'r', 'i', 'c', 'h', 't'], 'O'], ['werd', ['w', 'e', 'r', 'd'], 'O'], ['alvast', ['a', 'l', 'v', 'a', 's', 't'], 'O'], ['bekendgemaakt', ['b', 'e', 'k', 'e', 'n', 'd', 'g', 'e', 'm', 'a', 'a', 'k', 't'], 'O'], ['door', ['d', 'o', 'o', 'r'], 'O'], ['een', ['e', 'e', 'n'], 'O'], ['communicatiebureau', ['c', 'o', 'm', 'm', 'u', 'n', 'i', 'c', 'a', 't', 'i', 'e', 'b', 'u', 'r', 'e', 'a', 'u'], 'O'], [

In [21]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [22]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
# print((train_set[0][2]))

[[9565, 16394, 26650, 3603, 17306, 3311, 18255, 12598, 27489, 28364, 18348, 32821, 17306, 34298, 17814, 214, 3679, 21003, 7190, 2125, 32702, 30132, 4226, 31067], [2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4], array([[83,  2, 51, ...,  0,  0,  0],
       [ 2, 10,  0, ...,  0,  0,  0],
       [67, 10, 25, ...,  0,  0,  0],
       ...,
       [11, 88, 97, ...,  0,  0,  0],
       [37, 80,  3, ...,  0,  0,  0],
       [29,  0,  0, ...,  0,  0,  0]], dtype=int32), [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7]]
24


In [23]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [26]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3060        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [27]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch
        labels = labels.tolist()
#         print(labels)
        for sentence in labels:
            for i,label in enumerate(sentence):
                temp = [0]*len(label2Idx)
                value = label[0]
                temp[value]=1
                sentence[i] = temp
        labels = np.array(labels)
#         print(labels)
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
#         print('*')
        print(' ')

Epoch 0/50
139/85 [=================================================] - 35s 250ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 36s 52ms/step
 
453/85 [===============================================================================================================================================================] - 38s 84ms/step
 
155/85 [======================================================] - 40s 257ms/step
 
88/85 [===============================] - 40s 456ms/step                        
 
858/85 [=================================================================================================================================================================================================================================================================================

139/85 [=================================================] - 26s 187ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 28s 39ms/step
 
453/85 [===============================================================================================================================================================] - 29s 64ms/step
 
155/85 [======================================================] - 30s 192ms/step
 
88/85 [===============================] - 30s 342ms/step                        
 
858/85 [============================================================================================================================================================================================================================================================================================

155/85 [======================================================] - 31s 202ms/step
 
88/85 [===============================] - 32s 359ms/step                        
 
858/85 [==============================================================================================================================================================================================================================================================================================================] - 33s 39ms/step
 
653/85 [======================================================================================================================================================================================================================================] - 35s 54ms/step                                                                        
 
93/85 [================================] - 35s 377ms/step                                                                                                                     

139/85 [=================================================] - 27s 191ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 29s 40ms/step
 
453/85 [===============================================================================================================================================================] - 30s 66ms/step
 
155/85 [======================================================] - 32s 204ms/step
 
88/85 [===============================] - 32s 362ms/step                        
 
858/85 [============================================================================================================================================================================================================================================================================================

139/85 [=================================================] - 25s 181ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 27s 39ms/step
 
453/85 [===============================================================================================================================================================] - 28s 63ms/step
 
155/85 [======================================================] - 30s 196ms/step
 
88/85 [===============================] - 31s 349ms/step                        
 
858/85 [============================================================================================================================================================================================================================================================================================

...

 
453/85 [===============================================================================================================================================================] - 29s 64ms/step
 
155/85 [======================================================] - 30s 195ms/step
 
88/85 [===============================] - 30s 345ms/step                        
 
858/85 [==============================================================================================================================================================================================================================================================================================================] - 32s 37ms/step
 
653/85 [======================================================================================================================================================================================================================================] - 33s 51ms/step                                                            

 
147/85 [===================================================] - 37s 250ms/step
 
107/85 [=====================================] - 37s 345ms/step              
 
Epoch 45/50
139/85 [=================================================] - 27s 192ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 28s 40ms/step
 
453/85 [===============================================================================================================================================================] - 30s 66ms/step
 
155/85 [======================================================] - 32s 204ms/step
 
88/85 [===============================] - 32s 362ms/step                        
 
858/85 [==============================================================================================================

139/85 [=================================================] - 26s 185ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 28s 39ms/step
 
453/85 [===============================================================================================================================================================] - 29s 64ms/step
 
155/85 [======================================================] - 31s 199ms/step
 
88/85 [===============================] - 31s 353ms/step                        
 
858/85 [============================================================================================================================================================================================================================================================================================

 
155/85 [======================================================] - 27s 174ms/step
 
88/85 [===============================] - 27s 309ms/step                        
 
858/85 [==============================================================================================================================================================================================================================================================================================================] - 29s 33ms/step
 
653/85 [======================================================================================================================================================================================================================================] - 30s 45ms/step                                                                        
 
93/85 [================================] - 30s 319ms/step                                                                                                                   

139/85 [=================================================] - 20s 142ms/step
 
707/85 [=========================================================================================================================================================================================================================================================] - 22s 31ms/step
 
453/85 [===============================================================================================================================================================] - 23s 50ms/step
 
155/85 [======================================================] - 24s 155ms/step
 
88/85 [===============================] - 24s 275ms/step                        
 
858/85 [============================================================================================================================================================================================================================================================================================

In [28]:
predLabels, correctLabels = tag_dataset(dev_batch)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

5194/5195 [============================>.] - ETA: 0sDev-Data: Prec: 0.815, Rec: 0.799, F1: 0.807


In [29]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

2893/2895 [============================>.] - ETA: 0sTest-Data: Prec: 0.804, Rec: 0.789, F1: 0.796
